---
# Import Libraries

In [1]:
import os
import sys

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
sys.path.append("../../../deep-learning-dna")
sys.path.append("../")
sys.path.append("../../../deep-learning-dna/common")

In [4]:
import wandb

In [6]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import math
import string

from Attention import Set_Transformer
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utilities as tfu

In [7]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

---
# Load Data

In [8]:
def gen_data(batch_size, length=5):
    x = np.random.randint(1, 100, (batch_size, length))
    y = np.max(x, axis=1)
    return x, y # (batch_size, length)

In [9]:
x, y = gen_data(3, 5)

In [10]:
print(x.shape, y.shape)

(3, 5) (3,)


In [11]:
x

array([[93,  9, 62, 84, 35],
       [49, 79, 54,  5, 66],
       [64, 38, 93, 32, 64]])

In [12]:
y

array([93, 79, 93])

---
# Cache Memory

In [13]:
def Cache_Memory(current_state, previous_state, memory_length):
    
    if memory_length is None or memory_length == 0:
        return None
    else:
        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# Attention

In [14]:
class Attention(keras.Model):
    def __init__(self, num_induce, embed_dim, num_heads, use_layernorm, pre_layernorm, use_keras_mha):
        super(Attention, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        if self.num_induce == 0:       
            self.attention = (Set_Transformer.SetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm,use_keras_mha=self.use_keras_mha))
        else:
            self.attention = Set_Transformer.InducedSetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, num_induce=self.num_induce, use_layernorm=self.use_layernorm, pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha)
    
    
    def call(self, data, mems):
                
            attention = self.attention([data, mems])
                
            return attention

---
# XL Block

In [15]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 num_induce, 
                 embed_dim,
                 num_heads,
                 use_layernorm,
                 pre_layernorm,
                 use_keras_mha,):

        super(TransformerXLBlock, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.attention = Attention
        
        self.attention_layer = self.attention(self.num_induce, self.embed_dim, self.num_heads, self.use_layernorm, self.pre_layernorm, self.use_keras_mha)

   
    def call(self,
             content_stream,
             state=None):
        
        attention_output = self.attention_layer(content_stream, state)

        return attention_output

---
# Transformer XL

In [16]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 mem_switched, 
                 num_layers,
                 num_induce,
                 embed_dim,
                 num_heads,
                 dropout_rate,
                 mem_len=None,
                 use_layernorm=True,
                 pre_layernorm=True, 
                 use_keras_mha=True):
        
        super(TransformerXL, self).__init__()

        self.mem_switched = mem_switched
        self.num_layers = num_layers
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.mem_len = mem_len
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha

        self.transformer_xl_layers = []
        
        for i in range(self.num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(self.num_induce,
                                        self.embed_dim,
                                        self.num_heads,
                                        self.use_layernorm,
                                        self.pre_layernorm, 
                                        self.use_keras_mha))

        self.output_dropout = tf.keras.layers.Dropout(rate=self.dropout_rate)

    def call(self,
             content_stream,
             state=None):
        
        new_mems = []

        if state is None:
            state = [None] * self.num_layers
            
        for i in range(self.num_layers):
            if self.mem_switched == False:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))
            
            transformer_xl_layer = self.transformer_xl_layers[i]
            
            transformer_xl_output = transformer_xl_layer(content_stream=content_stream,
                                                        state=state[i])
            
            content_stream = self.output_dropout(transformer_xl_output)
            
            if self.mem_switched == True:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))

        output_stream = content_stream
        return output_stream, new_mems

---
# Xl Model Class

In [17]:
class XlModel(keras.Model):
    def __init__(self, mem_switched, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha):
        super(XlModel, self).__init__()
        
        self.mem_switched = mem_switched
        self.max_files = max_files
        self.encoder = encoder
        self.block_size = block_size
        self.max_set_len = max_set_len
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.mem_len = mem_len
        self.dropout_rate = dropout_rate
        self.num_seeds = num_seeds
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.linear_layer = keras.layers.Dense(self.embed_dim)
        
        self.transformer_xl = TransformerXL(self.mem_switched,
                                            self.num_layers,
                                             self.num_induce,
                                             self.embed_dim,
                                             self.num_heads,
                                             self.dropout_rate,
                                             self.mem_len,
                                             self.use_layernorm,
                                             self.pre_layernorm,
                                             self.use_keras_mha)
        

        self.segment_pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
       
        self.output_pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
    
        self.dropout_layer = keras.layers.Dropout(.5)
    
        self.dense_layer = keras.layers.Dense(1) 
        
        #Classification
        #self.dense_layer = keras.layers.Dense(100) 

    def call(self, x, training=None):    
 
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.mem_len, self.embed_dim))
    
        x = tf.expand_dims(x, axis=2)
        
        linear_transform = self.linear_layer(x)

        outputs = []
        
        for i in range(0, self.max_set_len, self.block_size):

            block = linear_transform[:,i:i+self.block_size]
            
            output, mems = self.transformer_xl(content_stream=block, state=mems)
            
            segment_pooling = self.segment_pooling_layer(output)

        output_pooling = self.output_pooling_layer(segment_pooling)
        
        dropout = self.dropout_layer(output_pooling)

        dense = self.dense_layer(dropout)

        #output = tf.reshape(dense, tf.shape(dense)[:2])
        
        #Classification
        output = tf.reshape(dense, tf.shape(dense)[::2])         
        
        return output

---
# Xl Parameters

In [18]:
#Xl Parameters
max_files = 0
encoder = 0
mem_switched = True
num_induce = 0
embed_dim = 64
num_layers = 4
num_heads = 8
mem_len = 0
dropout_rate = 0.01
num_seeds = 1
use_layernorm = True
pre_layernorm = True
use_keras_mha = True

---
# Create Models

In [19]:
block_size = 20
length = 20

In [20]:
x, y = gen_data(batch_size=10, length=length)
vx, vy = gen_data(batch_size=10, length=length)

In [21]:
max_set_len = length
set_len = length

In [22]:
model = XlModel(mem_switched, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha)
model.compile(optimizer=keras.optimizers.Adam(1e-3),loss=keras.losses.MeanAbsoluteError())

In [23]:
y[:5]

array([97, 97, 97, 99, 89])

In [24]:
model.predict(x[:5])

1/1 [==============================] - 5s 5s/step


array([[1.5858469],
       [1.6071842],
       [1.589977 ],
       [1.5967275],
       [1.6155479]], dtype=float32)

In [25]:
epochs = 500

In [41]:
history = model.fit(x, y, validation_data=(vx, vy), epochs=epochs, verbose=1)

Epoch 1/500
1/1 [==============================] - 0s 260ms/step - loss: 27.3071 - val_loss: 25.3461
Epoch 2/500
1/1 [==============================] - 0s 235ms/step - loss: 22.5488 - val_loss: 25.2865
Epoch 3/500
1/1 [==============================] - 0s 190ms/step - loss: 23.7848 - val_loss: 25.2290
Epoch 4/500
1/1 [==============================] - 0s 222ms/step - loss: 25.4851 - val_loss: 25.1684
Epoch 5/500
1/1 [==============================] - 0s 180ms/step - loss: 19.4116 - val_loss: 25.1043
Epoch 6/500
1/1 [==============================] - 0s 194ms/step - loss: 23.7229 - val_loss: 25.0390
Epoch 7/500
1/1 [==============================] - 0s 210ms/step - loss: 26.4854 - val_loss: 24.9770
Epoch 8/500
1/1 [==============================] - 0s 232ms/step - loss: 28.1708 - val_loss: 24.9142
Epoch 9/500
1/1 [==============================] - 0s 193ms/step - loss: 20.4622 - val_loss: 24.8553
Epoch 10/500
1/1 [==============================] - 0s 276ms/step - loss: 23.2211 - val_los

In [42]:
#Training

In [43]:
x[:5]

array([[ 1,  9,  7, 61, 34, 73, 35, 34, 79, 22, 97, 41, 42, 10, 65, 59,
        30, 39, 73, 17],
       [89, 84, 42, 48,  4,  4, 97, 16, 76, 48, 61, 58, 31, 93,  8, 48,
        68,  7, 50, 66],
       [71, 84,  4, 55, 90, 94, 56,  8, 97, 73, 17, 77,  2, 43, 64, 75,
        70, 56, 30, 33],
       [72, 81, 91, 68, 99, 99, 62, 61, 97,  5, 31, 36,  2, 84, 76, 60,
        40, 81, 61, 79],
       [66,  4,  3,  6, 56, 61, 57, 18, 72, 56, 81,  4, 83, 27, 89, 81,
        26, 54, 87, 28]])

In [44]:
y[:5]

array([97, 97, 97, 99, 89])

In [45]:
model.predict(x[:5])

1/1 [==============================] - 0s 172ms/step


array([[90.368835],
       [90.36876 ],
       [90.368774],
       [90.36879 ],
       [90.36872 ]], dtype=float32)

In [40]:
#tf.argmax(model.predict(x[:5]), axis = 1)

In [178]:
#Validation

In [179]:
x, y = gen_data(batch_size=30, length=10)

In [ ]:
y[:5]

array([90, 92, 76, 95, 81])

In [181]:
tf.argmax(model.predict(x[:5]), axis = 1)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>